In [10]:

import pandas as pd
import numpy as np 
!pip install requests
import requests 
import shutil 
import multiprocessing

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'd:\python.exe -m pip install --upgrade pip' command.


In [12]:
# chemin base de données avec url image



C:\Users\Utilisateur\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [13]:
df.head()


,Unnamed: 0,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,countries,countries_tags,countries_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url
0,20,000000000114,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1580066482,2020-01-26T19:21:22Z,1609945229,2021-01-06T15:00:29Z,Chocolate n 3,en:fr,en:france,France,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,NaN,NaN,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...
1,22,0000000001199,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1517833594,2018-02-05T12:26:34Z,1540674511,2018-10-27T21:08:31Z,Solène céréales poulet,France,en:france,France,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...
2,23,0000000001281,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1517830801,2018-02-05T11:40:01Z,1527070794,2018-05-23T10:19:54Z,Tarte noix de coco,France,en:france,France,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,NaN,NaN
3,30,0000000001663,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1509478049,2017-10-31T19:27:29Z,1509478053,2017-10-31T19:27:33Z,Crème dessert chocolat,France,en:france,France,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,NaN,NaN
4,33,0000000001885,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1511180337,2017-11-20T12:18:57Z,1518126491,2018-02-08T21:48:11Z,Compote de poire,France,en:france,France,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,NaN,NaN,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...


In [ ]:
class myThread (threading.Thread):
    def __init__(self, threadID, name, q):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.q = q
    
    def run(self):
        print ("Starting " + self.name)
        process_data(self.name, self.q)
        print ("Exiting " + self.name)

In [ ]:
import queue
import threading
import time

list_url = df['image_url'].to_numpy()
list_code = df['code'].to_numpy()
exitFlag = 0

# fonction pour télécharger une image
def process_data(threadName, q):
    while not exitFlag:
        queueLock.acquire()
        if not workQueue.empty():
            loic, nom = q.get()
        queueLock.release()
        r = requests.get(loic, stream=True)
        if r.status_code != 200:
            with open("image_non_telecharger.txt", "w") as fichier:
                fichier.write(f"image not télécharger code : {nom}\n")
        filename = "./data_image/" + str(nom) + "." +  loic.split('.')[-1]
        with open(filename, 'wb') as f:
            shutil.copyfileobj(r.raw, f)

threadList = ["Thread-1", "Thread-2", "Thread-3", "Thread-4", "Thread-5", "Thread-6", "Thread-7", "Thread-8"]
queueLock = threading.Lock()
queueLock.acquire()
workQueue = queue.Queue(len(list_code))
threads = []
threadID = 1

print("Create new threads")
# Create new threads
for tName in threadList:
    thread = myThread(threadID, tName, workQueue)
    thread.start()
    threads.append(thread)
    threadID += 1

print("Fill the queue")
# Fill the queue
#queueLock.acquire()
for coup in zip(list_url[65536:], list_code[65536:]):
    workQueue.put(coup)
queueLock.release()

print("Wait the queue")
# Wait for queue to empty
while not workQueue.empty():
    pass

# Notify threads it's time to exit
exitFlag = 1

# Wait for all threads to complete
for t in threads:
    t.join()
print ("Exiting Main Thread")

In [15]:
list_url = df['image_url'].to_numpy()[0:1000]
list_code = df['code'].to_numpy()[0:1000]
inputs = list(zip(list_url, list_code))


def multiprocessing_func(x):
    """Individually prints the squares y_i of the elements x_i of a vector x"""
    print(x)
    r = requests.get(x[0], stream=True)
    if r.status_code != 200:
        with open("image_non_telecharger.txt", "w") as fichier:
            fichier.write(f"image not télécharger code : {x[1]}\n")
    else:
        filename = "./data_image_2/" + str(x[1]) + "." + x[0].split('.')[-1]
        with open(filename, 'wb') as f:
            shutil.copyfileobj(r.raw, f)


def chunks(input, n):
    """Yields successive n-sized chunks of input"""
    for i in range(0, len(inputs), n):
        yield input[i:i + n]



    print("fini")

if __name__ == '__main__':
    n_proc = 6
    x = np.arange(100)
    chunked_x = list(chunks(x, int(x.shape[0]/n_proc)+1))
    processes = []
    for i in np.arange(0, n_proc):
        p = multiprocessing.Process(target=multiprocessing_func, args=(chunked_x[i],))
        processes.append(p)
        p.start()
    for process in processes:
        process.join()
    print("salut")


fini


fini
salut
